In [72]:
import pandas as pd
import numpy as np
from Bio.Seq import Seq
from Bio import SeqIO
#from Bio.Alphabet import IUPAC
import sys
# import pyensembl
import os
#https://towardsdatascience.com/a-simple-guide-to-command-line-arguments-with-argparse-6824c30ab1c3
import warnings
warnings.filterwarnings("ignore")
from Bio.SeqUtils import seq3

import glob
import protfasta
import Levenshtein


In [74]:
# SK: Dictionary of protein names to sequences
proteins = {}
for record in SeqIO.parse("../raw_files/gencode.v36.pc_translations.fa", "fasta"):
    name = record.id.split("|")[1].split(".")[0]
    proteins[name] = str(record.seq)
    

# SK: Dictionary of protein names to CDS dna transcript

dna_transcripts = {}
for record in SeqIO.parse("../raw_files/gencode.v36.pc_transcripts.fa", "fasta"):
	name = record.id.split("|")[0].split(".")[0]
	record_c = record.id.split("|")
	for i in record_c:
		if "CDS" in i:
			coords = i.replace("CDS:","")
	start = int(coords.split("-")[0])
	end = int(coords.split("-")[1])
	dna_seq = str(record.seq)[start-1:end]
	dna_transcripts[name] = dna_seq
    
uniprotID_ENST_mapping = pd.read_csv("../../data/SFARI_TFs_with_ENST_corrected.csv")
uniprotID_ENST_mapping = uniprotID_ENST_mapping[["uniprotID", "ENST"]]
uniprotID_ENST_mapping["ENST"] = uniprotID_ENST_mapping["ENST"].str.split(".").str[0]
uniprotID_ENST_mapping_dict= dict(zip(uniprotID_ENST_mapping["uniprotID"],uniprotID_ENST_mapping["ENST"]))
uniprotID_ENST_mapping_dict['O60479'] = 'ENST00000434704'

In [92]:
directory = "../outputs/mutations/domains_bed_format/"
files = os.listdir(directory)

In [26]:
def save_variant_fasta(uniprotID, variant_file_name = "expanded_iWES_v2_variants", save = True, variants_df = []):
    if uniprotID in uniprotID_ENST_mapping_dict.keys():
        # finding ENST corresponding to uniprotID, reading in missense SNP variants
        ENST = uniprotID_ENST_mapping_dict[uniprotID]
        print("../outputs/mutations/domains_" + variant_file_name + "_snv_classified/" + ENST + ".bed")

        if len(variants_df) == 0:
            try:
                variants = pd.read_csv("../outputs/mutations/domains_" + variant_file_name + "_snv_classified/" + ENST + ".bed", sep = "\t", header = None)
            except pd.errors.EmptyDataError:
                print(ENST, " has no variants.")
                variants = pd.DataFrame()
        else:
            variants = variants_df
            # must have:
            # columns named:
                # 22: with mutation type (ie "No-Syn", "Syn", etc)
                # 3: domain type (ie "AD" or "DBD")
                # 13: strand
                # 2: position (1-indexed)
                # 17: Reference nt
                # 0: chromosome
                # 18: alternate nt

        if len(variants) != 0:
            
            print(ENST)
            display(variants)
            no_syn_variants = variants[variants[22] == "No-Syn"]
            no_syn_variants = no_syn_variants[no_syn_variants[3] == "AD"]

            # Finding strand
            strand = no_syn_variants[13].iloc[0]
    
            # Finding wt nucleotide sequence of transcript, and corresponding AA sequence
            wt_nt_seq = dna_transcripts[ENST]
            wt_AA_seq = str(Seq(wt_nt_seq).translate())
            display(wt_AA_seq)
    
            if strand == "-":
                wt_nt_seq = str(Seq(wt_nt_seq).complement())
    
            nt_df = pd.DataFrame({"nt" : [*wt_nt_seq]})
            nt_df["pred_prot_pos"] = [i for i in range(1, int((len(nt_df)) / 3 + 1)) for _ in range(3)]
        
            cds_bed = pd.read_csv("../outputs/mutations/cds_bed_format/" + ENST, sep = "\t", header = None)
            cds_bed = cds_bed[[1, 2]]
            cds_bed[1] += 1
    
            if strand == "-":
                cds_bed = cds_bed.sort_values(by = 1, ascending = False)
            else:
                cds_bed = cds_bed.sort_values(by = 1, ascending = True)
    
            range_col = []
            for start, end in zip(cds_bed[1], cds_bed[2]):
                if strand == "-":
                    range_col += list(range(end, start - 1, -1))
                else:
                    range_col += list(range(start, end + 1))
    
            nt_df["gen_pos"] = range_col
            nt_df = nt_df.set_index("gen_pos")
            display(nt_df)
    
            # Checking 
            for i in no_syn_variants.index:
                pos = no_syn_variants[2].loc[i]
                sfari_nt = no_syn_variants[no_syn_variants[2] == pos][17].loc[i]
                nt_df_nt = nt_df.loc[pos].iloc[0]
                if sfari_nt != nt_df_nt:
                    print("mismatch!")
    
            # display(no_syn_variants)
            no_syn_variants["name"] = "g." + no_syn_variants[0].astype(str) + "."+ no_syn_variants[2].astype(str) + no_syn_variants[17] + ">" + no_syn_variants[18]
            
            names = []
            TF_seqs = []
            prot_positions = []
    
    
            for i in no_syn_variants.index:
                var_pos = no_syn_variants[2].loc[i]
                wt_nt = no_syn_variants[17].loc[i]
                var_nt = no_syn_variants[18].loc[i]
                g_name = no_syn_variants["name"].loc[i]
    
                nt_df_var_copy = nt_df.copy(deep = True)
    
                if nt_df_var_copy.at[var_pos, "nt"] == wt_nt:
                    nt_df_var_copy.at[var_pos, "nt"] = var_nt
                else:
                    print(uniprotID + " mismatch!")
    
                new_nt_seq = "".join(nt_df_var_copy["nt"])
                if strand == "-":
                    new_nt_seq = Seq(new_nt_seq).complement()
                new_AA_seq = str(Seq(new_nt_seq).translate())
    
                # prot_pos = 0
                for i in range(len(new_AA_seq)):
                    if new_AA_seq[i] != wt_AA_seq[i]:
                        prot_change_descrip = "p." + seq3(wt_AA_seq[i]) + str(i + 1)+ seq3(new_AA_seq[i])
                        prot_positions.append(i + 1)
                        names.append(g_name + "(" + prot_change_descrip + ")")
                        TF_seqs.append(new_AA_seq)
    
                    # prot_pos = i + 1    
    
                    
                
            variant_TF_seqs_df = pd.DataFrame({"name": names,
                         "TF_seq": TF_seqs, 
                        "prot_pos":prot_positions})
    
            def return_AD_seqs(AD_start, AD_end):
                print(AD_start)
                print(AD_end)
                #display(variant_TF_seqs_df)
                within_AD = variant_TF_seqs_df[(AD_start <= variant_TF_seqs_df["prot_pos"]) & (variant_TF_seqs_df["prot_pos"] <= AD_end)]
                #display(within_AD)
                within_AD["AD_seq"] = within_AD["TF_seq"].str[AD_start - 1: AD_end]
                #display(within_AD)
                return within_AD
    
            # return_AD_seqs(340, 477)
    
            known_ADs = pd.read_csv("../../output/known_ADs_considering_isoforms_and_canonical.csv")
            AD_rows = known_ADs[known_ADs["uniprotID"] == uniprotID]
    
            gene = AD_rows["Gene"].iloc[0].replace(" ", "").replace("/", "-")
    
            def save_fasta(start, end):
                df = return_AD_seqs(start, end)
                df = df.reset_index()
                display(df)
                ofile = open("../outputs/AD_" + variant_file_name + "_variant_fasta/" + gene + "_" + uniprotID + "_" + "AD_" + str(start) + "-" + str(end), "w")
                for i in df.index:
                    ofile.write(">" + df["name"].loc[i] + "\n" + df["AD_seq"].loc[i] + "\n")
                ofile.close()
                
        
            # display(AD_rows)
    
            if save:
                for start, end in zip(AD_rows["Start"], AD_rows["End"]):
                    save_fasta(start, end)
            
            return variant_TF_seqs_df


---
### Using to find gnomAD variants in ADs

In [31]:
for uniprotID in list(files):
    save_variant_fasta(uniprotID, variant_file_name = "gnomad", save = True)

../outputs/mutations/domains_gnomad_snv_classified/ENST00000592199.bed
ENST00000592199


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,19,12995841,12995842,DBD,ENSG00000008441,.,+,ENST00000592199,19,12995841,...,+,19,12995841,12995842,A,G,0.000051,Y,C,No-Syn
1,19,12995843,12995844,DBD,ENSG00000008441,.,+,ENST00000592199,19,12995843,...,+,19,12995843,12995844,T,C,0.000001,S,P,No-Syn
2,19,12995844,12995845,DBD,ENSG00000008441,.,+,ENST00000592199,19,12995844,...,+,19,12995844,12995845,C,A,0.000001,S,Y,No-Syn
3,19,12995845,12995846,DBD,ENSG00000008441,.,+,ENST00000592199,19,12995845,...,+,19,12995845,12995846,C,T,0.000011,S,S,Syn
4,19,12995846,12995847,DBD,ENSG00000008441,.,+,ENST00000592199,19,12995846,...,+,19,12995846,12995847,C,G,0.000007,P,A,No-Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,19,13094635,13094636,AD,ENSG00000008441,.,+,ENST00000592199,19,13094635,...,+,19,13094635,13094636,C,A,0.000002,S,Y,No-Syn
318,19,13094639,13094640,AD,ENSG00000008441,.,+,ENST00000592199,19,13094639,...,+,19,13094639,13094640,G,A,0.000001,W,*,NoSense
319,19,13094642,13094643,AD,ENSG00000008441,.,+,ENST00000592199,19,13094642,...,+,19,13094642,13094643,C,T,0.000004,F,F,Syn
320,19,13094644,13094645,AD,ENSG00000008441,.,+,ENST00000592199,19,13094644,...,+,19,13094644,13094645,T,C,0.000007,L,P,No-Syn


'MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEKRMSKDEERAVKDELLGEKPEIKQKWASRLLAKLRKDIRPEFREDFVLTITGKKPPCCVLSNPDQKGKIRRIDCLRQADKVWRLDLVMVILFKGIPLESTDGERLYKSPQCSNPGLCVQPHHIGVTIKELDLYLAYFVHTPESGQSDSSNQQGDADIKPLPNGHLSFQDCFVTSGVWNVTELVRVSQTPVATASGPNFSLADLESPSYYNINQVTLGRRSITSPPSTSTTKRPKSIDDSEMESPVDDVFYPGTGRSPAAGSSQSSGWPNDVDAGPASLKKSGKLDFCSALSSQGSSPRMAFTHHPLPVLAGVRPGSPRATASALHFPSTSIIQQSSPYFTHPTIRYHHHHGQDSLKEFVQFVCSDGSGQATGQPNGSGQGKVPGSFLLPPPPPVARPVPLPMPDSKSTSTAPDGAALTPPSPSFATTGASSANRFVSIGPRDGNFLNIPQQSQSWFL*'

,nt,pred_prot_pos
gen_pos,,
12995838,A,1
12995839,T,1
12995840,G,1
12995841,T,2
12995842,A,2
...,...,...
13094645,T,502
13094646,C,502
13094647,T,503


414
502


...

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,2,1922495,1922496,AD,ENSG00000186487,.,-,ENST00000428368,2,1922495,...,-,2,1922495,1922496,C,T,4.059529e-06,D,N,No-Syn
1,2,1922495,1922496,AD,ENSG00000186487,.,-,ENST00000428368,2,1922495,...,-,2,1922495,1922496,C,A,1.200322e-06,D,Y,No-Syn
2,2,1922496,1922497,AD,ENSG00000186487,.,-,ENST00000428368,2,1922496,...,-,2,1922496,1922497,G,A,4.337916e-06,F,F,Syn
3,2,1922496,1922497,AD,ENSG00000186487,.,-,ENST00000428368,2,1922496,...,-,2,1922496,1922497,G,T,6.841312e-07,F,L,No-Syn
4,2,1922502,1922503,AD,ENSG00000186487,.,-,ENST00000428368,2,1922502,...,-,2,1922502,1922503,C,T,1.200322e-06,E,E,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,2,1943021,1943022,AD,ENSG00000186487,.,-,ENST00000428368,2,1943021,...,-,2,1943021,1943022,T,C,3.010638e-06,E,E,Syn
394,2,1943024,1943025,AD,ENSG00000186487,.,-,ENST00000428368,2,1943024,...,-,2,1943024,1943025,C,T,1.776432e-06,E,E,Syn
395,2,1943026,1943027,AD,ENSG00000186487,.,-,ENST00000428368,2,1943026,...,-,2,1943026,1943027,C,A,7.341228e-07,E,*,NoSense
396,2,1943027,1943028,AD,ENSG00000186487,.,-,ENST00000428368,2,1943027,...,-,2,1943027,1943028,T,C,7.365354e-07,E,E,Syn


'MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKYARHRSVYGCPLAKKRKTQDKQPQEPAPKRKPFAVKADSSSVDECDDSDGTEDMDEKEEDEGEEYSEDNDEPGDEDEEDEEGDREEEEEIEEEDEDDDEDGEDVEDEEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNYDELVAKSLLNLGKIAEDAAYRARTESEMNSNTSNSLEDDSDKNENLGRKSELSLDLDSDVVRETVDSLKLLAQGHGVVLSENMNDRNYADSMSQQDSRNMNYVMLGKPMNNGLMEKMVEESDEEVCLSSLECLRNQCFDLARKLSETNPQERNPQQNMNIRQHVRPEEDFPGRTPDRNYSDMLNLMRLEEQLSPRSRVFASCAKEDGCHERDDDTTSVNSDRSEEVFDMTKGNLTLLEKAIALETERAKAMREKMAMEAGRRDNMRSYEDQSPRQLPGEDRKPKSSDSHVKKPYYGKDPSRTEKKESKCPTPGCDGTGHVTGLYPHHRSLSGCPHKDRVPPEILAMHESVLKCPTPGCTGRGHVNSNRNSHRSLSGCPIAAAEKLAKAQEKHQSCDVSKSSQASDRVLRPMCFVKQLEIPQYGYRNNVPTTTPRSNLAKELEKYSKTSFEYNSYDNHTYGKRAIAPKVQTRDISPKGYDDAKRYCKDPSPSSSSTSSYAPSSSSNLSCGGGSSASSTCSKSSFDYTHDMEAAHMAATAILNLSTRCREMPQNLSTKPQDLCATRNPDMEVDENGTLDLSMNKQRPRDSCCPILTPLEPMSPQQQAVMNNRCFQLGEGDCWDLPVDYTKMKPRRIDEDESKDITPEDLDPFQEALEERRYPGEVTIPSPKPKYPQCKESKKDLITLSGCPLADKSIRSMLATSSQELKCPTPGCDGSGHITGNYASHRSLSGCPRAKKSGIRIAQSKEDKEDQEPIRCPVPGCDGQGHITGKYASHRSASGCPLAAKRQKDGYLNGSQFSWKS

,nt,pred_prot_pos
gen_pos,,
1979777,T,1
1979776,A,1
1979775,C,1
1979774,C,2
1979773,T,2
...,...,...
1791871,A,1186
1791870,G,1186
1791869,A,1187


152
425


,index,name,TF_seq,prot_pos,AD_seq
0,0,g.2.1922496C>T(p.Asp425Asn),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,425,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
1,1,g.2.1922496C>A(p.Asp425Tyr),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,425,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
2,2,g.2.1922497G>T(p.Phe424Leu),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,424,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
3,3,g.2.1922505C>T(p.Glu422Lys),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,422,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
4,4,g.2.1922510G>C(p.Ser420Cys),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,420,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
...,...,...,...,...,...
251,251,g.2.1943006C>G(p.Glu161Gln),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,161,EEEEEEEEEQEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
252,252,g.2.1943010C>A(p.Glu159Asp),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,159,EEEEEEEDEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
253,253,g.2.1943012C>T(p.Glu159Lys),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,159,EEEEEEEKEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
254,254,g.2.1943012C>G(p.Glu159Gln),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,159,EEEEEEEQEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...


In [32]:
# Checking work - distance at most 1 from known AD?

known_ADs = pd.read_csv("../../output/known_ADs_considering_isoforms_and_canonical.csv")

variant_ads = glob.glob("../outputs/AD_gnomad_variant_fasta/*")

for variant_AD in variant_ads:
    print(variant_AD)
    # Extracting info about AD (name and position)
    AD_descrip = variant_AD.split("/")[-1]
    gene_name = AD_descrip.split("_")[0]
    coords = AD_descrip.split("_")[-1]
    start = int(coords.split("-")[0])
    end = int(coords.split("-")[1])
    
    # Extracting corresponding wt AD seq
    entry = known_ADs[(known_ADs["Gene"] == gene_name) & (known_ADs["Start"] == start) & (known_ADs["End"] == end)]
    if len(entry) == 0:
        print("ERROR: AD not found")
    else:
        wt_seq = entry["ProteinRegionSeq"].iloc[0]
        
        # Finding variant sequences from fasta
        for var_seq in list(protfasta.read_fasta(variant_AD).values()):
            # Comparing distance
            dist = Levenshtein.distance(wt_seq, var_seq)
            #print(wt_seq)
            #print(first_var)
            
            if (dist != 1):
                print("DIFF > 1")
                print(dist)
                #print(wt_seq)
                #print(var_seq)
                print("----")

# As expected, only ERG different. 9/23/24: FIXED

../outputs/AD_gnomad_variant_fasta/GLIS1_Q8NBF1_AD_447-620
../outputs/AD_gnomad_variant_fasta/RORA_P35398_AD_272-385
../outputs/AD_gnomad_variant_fasta/VDR_P11473_AD_415-427
../outputs/AD_gnomad_variant_fasta/SRCAP_Q6ZRS2_AD_2575-3230
../outputs/AD_gnomad_variant_fasta/THRA_P10827_AD_1-52
../outputs/AD_gnomad_variant_fasta/YY1_P25490_AD_371-397
../outputs/AD_gnomad_variant_fasta/NCOA1_Q15788_AD_840-1011
../outputs/AD_gnomad_variant_fasta/SRCAP_Q6ZRS2_AD_198-1445
../outputs/AD_gnomad_variant_fasta/ERG_P11308_AD_118-261
../outputs/AD_gnomad_variant_fasta/CAMTA2_O94983_AD_472-581
../outputs/AD_gnomad_variant_fasta/ERG_P11308_AD_433-479
../outputs/AD_gnomad_variant_fasta/EBF3_Q9H4W6_AD_281-360
../outputs/AD_gnomad_variant_fasta/MSX2_P35548_AD_234-267
../outputs/AD_gnomad_variant_fasta/YY1_P25490_AD_1-69
../outputs/AD_gnomad_variant_fasta/MTF1_Q14872_AD_510-624
../outputs/AD_gnomad_variant_fasta/OTX1_P32242_AD_172-354
../outputs/AD_gnomad_variant_fasta/ARX_Q96QS3_AD_472-562
../outputs/AD_gn

---
### Using to find AM variants in each AD

First, preparing AM variant df to fit format needed

In [36]:
top_10_am = pd.read_csv("../../output/am_10_most_pathogenic_15ADs.csv")
top_10_am

,Unnamed: 0,filepath,gene,uniprotID,start,end,ENST_x,Gene,range,foreign_key,...,ALT,genome,uniprot_id,transcript_id,protein_variant,am_pathogenicity,am_class,ENST_y,AA_pos,AA_orig
0,351,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,273,O95096_273,...,T,hg38,O95096,ENST00000377142.5,W273R,0.9918,likely_pathogenic,ENST00000377142,273.0,W
1,350,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,273,O95096_273,...,G,hg38,O95096,ENST00000377142.5,W273R,0.9918,likely_pathogenic,ENST00000377142,273.0,W
2,338,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,271,O95096_271,...,T,hg38,O95096,ENST00000377142.5,W271R,0.9836,likely_pathogenic,ENST00000377142,271.0,W
3,337,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,271,O95096_271,...,G,hg38,O95096,ENST00000377142.5,W271R,0.9836,likely_pathogenic,ENST00000377142,271.0,W
4,333,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,271,O95096_271,...,A,hg38,O95096,ENST00000377142.5,W271C,0.9545,likely_pathogenic,ENST00000377142,271.0,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1165,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,349,P32242_349,...,C,hg38,P32242,ENST00000282549.7,W349R,0.9983,likely_pathogenic,ENST00000282549,349.0,W
146,30,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,177,P32242_177,...,C,hg38,P32242,ENST00000282549.7,W177R,0.9965,likely_pathogenic,ENST00000282549,177.0,W
147,29,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,177,P32242_177,...,A,hg38,P32242,ENST00000282549.7,W177R,0.9965,likely_pathogenic,ENST00000282549,177.0,W
148,1179,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,351,P32242_351,...,G,hg38,P32242,ENST00000282549.7,F351C,0.9962,likely_pathogenic,ENST00000282549,351.0,F


In [38]:
# must have:
# columns named: 
    # 22: with mutation type (ie "No-Syn", "Syn", etc)
    # 3: domain type (ie "AD" or "DBD")
    # 2: position (1-indexed)
    # 17: Reference nt
     # 18: alternate nt

# 13: strand
# 0: chromosome
               

In [40]:
    # 22: with mutation type (ie "No-Syn", "Syn", etc)
    # 3: domain type (ie "AD" or "DBD")
    # 2: position (1-indexed)
    # 17: Reference nt
     # 18: alternate nt

top_10_am[22] = "No-Syn"
top_10_am[3] = "AD"
top_10_am[2] = top_10_am["POS"].astype(int)
top_10_am[17] = top_10_am["REF"]
top_10_am[18] = top_10_am["ALT"]
top_10_am

,Unnamed: 0,filepath,gene,uniprotID,start,end,ENST_x,Gene,range,foreign_key,...,am_pathogenicity,am_class,ENST_y,AA_pos,AA_orig,22,3,2,17,18
0,351,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,273,O95096_273,...,0.9918,likely_pathogenic,ENST00000377142,273.0,W,No-Syn,AD,21511928,A,T
1,350,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,273,O95096_273,...,0.9918,likely_pathogenic,ENST00000377142,273.0,W,No-Syn,AD,21511928,A,G
2,338,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,271,O95096_271,...,0.9836,likely_pathogenic,ENST00000377142,271.0,W,No-Syn,AD,21511934,A,T
3,337,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,271,O95096_271,...,0.9836,likely_pathogenic,ENST00000377142,271.0,W,No-Syn,AD,21511934,A,G
4,333,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,271,O95096_271,...,0.9545,likely_pathogenic,ENST00000377142,271.0,W,No-Syn,AD,21511932,C,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1165,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,349,P32242_349,...,0.9983,likely_pathogenic,ENST00000282549,349.0,W,No-Syn,AD,63056296,T,C
146,30,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,177,P32242_177,...,0.9965,likely_pathogenic,ENST00000282549,177.0,W,No-Syn,AD,63055780,T,C
147,29,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,177,P32242_177,...,0.9965,likely_pathogenic,ENST00000282549,177.0,W,No-Syn,AD,63055780,T,A
148,1179,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,351,P32242_351,...,0.9962,likely_pathogenic,ENST00000282549,351.0,F,No-Syn,AD,63056303,T,G


In [44]:
# Building df of uniprotID, chr, and strand

addit_info = top_10_am[["uniprotID"]].drop_duplicates()
chromosomes = []
strands = []

for uniprotID in addit_info["uniprotID"]:
    df = pd.read_csv("../outputs/mutations/domains_bed_format/" + uniprotID, sep = "\t", header = None)
    chromosomes.append(df.iloc[0][0])
    strands.append(df.iloc[0][6])

addit_info[0] = chromosomes
addit_info[13] = strands
addit_info

,uniprotID,0,13
0,O95096,20,-
10,P26367,11,-
20,Q15788,2,+
30,P11308,21,-
40,O14770,15,-
60,P78337,5,-
70,O94983,17,-
100,P43354,2,-
120,Q13422,7,+
130,Q02548,9,-


In [46]:
top_10_am = pd.merge(top_10_am, addit_info, on = "uniprotID", how = "left")
top_10_am

,Unnamed: 0,filepath,gene,uniprotID,start,end,ENST_x,Gene,range,foreign_key,...,ENST_y,AA_pos,AA_orig,22,3,2,17,18,0,13
0,351,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,273,O95096_273,...,ENST00000377142,273.0,W,No-Syn,AD,21511928,A,T,20,-
1,350,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,273,O95096_273,...,ENST00000377142,273.0,W,No-Syn,AD,21511928,A,G,20,-
2,338,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,271,O95096_271,...,ENST00000377142,271.0,W,No-Syn,AD,21511934,A,T,20,-
3,337,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,271,O95096_271,...,ENST00000377142,271.0,W,No-Syn,AD,21511934,A,G,20,-
4,333,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,271,O95096_271,...,ENST00000377142,271.0,W,No-Syn,AD,21511932,C,A,20,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1165,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,349,P32242_349,...,ENST00000282549,349.0,W,No-Syn,AD,63056296,T,C,2,+
146,30,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,177,P32242_177,...,ENST00000282549,177.0,W,No-Syn,AD,63055780,T,C,2,+
147,29,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,177,P32242_177,...,ENST00000282549,177.0,W,No-Syn,AD,63055780,T,A,2,+
148,1179,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,351,P32242_351,...,ENST00000282549,351.0,F,No-Syn,AD,63056303,T,G,2,+


In [48]:
for uniprotID in set(top_10_am["uniprotID"]):
    print(uniprotID)
    variants_df = top_10_am[top_10_am["uniprotID"] == uniprotID]
    #display(variants_df)
    save_variant_fasta(uniprotID, variant_file_name = "alphamissense_top10", save = True, variants_df = variants_df)

O14770
../outputs/mutations/domains_alphamissense_top10_snv_classified/ENST00000561208.bed
ENST00000561208


,Unnamed: 0,filepath,gene,uniprotID,start,end,ENST_x,Gene,range,foreign_key,...,ENST_y,AA_pos,AA_orig,22,3,2,17,18,0,13
40,6,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,340,O14770_340,...,ENST00000561208,340.0,D,No-Syn,AD,36896646,C,G,15,-
41,4,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,340,O14770_340,...,ENST00000561208,340.0,D,No-Syn,AD,36896645,T,A,15,-
42,3,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,340,O14770_340,...,ENST00000561208,340.0,D,No-Syn,AD,36896645,T,C,15,-
43,2,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,340,O14770_340,...,ENST00000561208,340.0,D,No-Syn,AD,36896645,T,G,15,-
44,17,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,342,O14770_342,...,ENST00000561208,342.0,S,No-Syn,AD,36896640,A,G,15,-
45,7,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,340,O14770_340,...,ENST00000561208,340.0,D,No-Syn,AD,36896646,C,A,15,-
46,5,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,340,O14770_340,...,ENST00000561208,340.0,D,No-Syn,AD,36896646,C,T,15,-
47,15,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,342,O14770_342,...,ENST00000561208,342.0,S,No-Syn,AD,36896639,G,A,15,-
48,20,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,343,O14770_343,...,ENST00000561208,343.0,N,No-Syn,AD,36896635,A,C,15,-
49,19,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,343,O14770_343,...,ENST00000561208,343.0,N,No-Syn,AD,36896635,A,T,15,-


'MAQRYDELPHYGGMDGVGVPASMYGDPHAPRPIPPVHHLNHGPPLHATQHYGAHAPHPNVMPASMGSAVNDALKRDKDAIYGHPLFPLLALVFEKCELATCTPREPGVAGGDVCSSDSFNEDIAVFAKQVRAEKPLFSSNPELDNLMIQAIQVLRFHLLELEKVHELCDNFCHRYISCLKGKMPIDLVIDERDGSSKSDHEELSGSSTNLADHNPSSWRDHDDATSTHSAGTPGPSSGGHASQSGDNSSEQGDGLDNSVASPGTGDDDDPDKDKKRQKKRGIFPKVATNIMRAWLFQHLTHPYPSEEQKKQLAQDTGLTILQVNNWFINARRRIVQPMIDQSNRAGFLLDPSVSQGAAYSPEGQPMGSFVLDGQQHMGIRPAGLQSMPGDYVSQGGPMGMSMAQPSYTPPQMTPHPTQLRHGPPMHSYLPSHPHHPAMMMHGGPPTHPGMTMSAQSPTMLNSVDPNVGGQVMDIHAQ*'

,nt,pred_prot_pos
gen_pos,,
37099466,T,1
37099465,A,1
37099464,C,1
37099463,C,2
37099462,G,2
...,...,...
36892177,T,477
36892176,T,477
36892175,A,478


340
477


...

,index,name,TF_seq,prot_pos,AD_seq


1241
1385


,index,name,TF_seq,prot_pos,AD_seq
0,0,g.2.24758047T>A(p.Val1319Asp),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1319,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
1,1,g.2.24758041T>C(p.Ile1317Thr),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1317,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
2,2,g.2.24758041T>A(p.Ile1317Asn),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1317,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
3,3,g.2.24758041T>G(p.Ile1317Ser),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1317,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
4,4,g.2.24758039C>A(p.Ser1316Arg),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1316,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
5,5,g.2.24758029A>T(p.Asn1313Ile),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1313,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
6,6,g.2.24758039C>G(p.Ser1316Arg),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1316,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
7,7,g.2.24758037A>C(p.Ser1316Arg),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1316,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
8,8,g.2.24758032A>T(p.Asn1314Ile),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1314,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
9,9,g.2.24758047T>G(p.Val1319Gly),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1319,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...


840
1011


,index,name,TF_seq,prot_pos,AD_seq
0,10,g.2.24728397T>C(p.Leu936Pro),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,936,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
1,11,g.2.24728454T>C(p.Leu955Pro),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,955,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
2,12,g.2.24728442T>C(p.Leu951Pro),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,951,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
3,13,g.2.24728450G>C(p.Ala954Pro),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,954,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
4,14,g.2.24728445A>T(p.Asp952Val),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,952,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
5,15,g.2.24728409T>C(p.Leu940Pro),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,940,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
6,16,g.2.24728385T>C(p.Leu932Pro),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,932,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
7,17,g.2.24728454T>A(p.Leu955Gln),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,955,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
8,18,g.2.24728388T>C(p.Leu933Pro),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,933,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
9,19,g.2.24728444G>C(p.Asp952His),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,952,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...


In [49]:
# Checking work - distance at most 1 from known AD?

variant_ads = glob.glob("../outputs/AD_alphamissense_top10_variant_fasta/*")

for variant_AD in variant_ads:
    print(variant_AD)
    # Extracting info about AD (name and position)
    AD_descrip = variant_AD.split("/")[-1]
    gene_name = AD_descrip.split("_")[0]
    coords = AD_descrip.split("_")[-1]
    start = int(coords.split("-")[0])
    end = int(coords.split("-")[1])
    
    # Extracting corresponding wt AD seq
    entry = known_ADs[(known_ADs["Gene"] == gene_name) & (known_ADs["Start"] == start) & (known_ADs["End"] == end)]
    if len(entry) == 0:
        print("ERROR: AD not found")
    else:
        wt_seq = entry["ProteinRegionSeq"].iloc[0]
        
        # Finding variant sequences from fasta
        for var_seq in list(protfasta.read_fasta(variant_AD).values()):
            # Comparing distance
            dist = Levenshtein.distance(wt_seq, var_seq)
            #print(wt_seq)
            #print(first_var)
            
            if (dist != 1):
                print("DIFF > 1")
                print(dist)
                #print(wt_seq)
                #print(var_seq)
                print("----")

# As expected, only ERG different.

../outputs/AD_alphamissense_top10_variant_fasta/NCOA1_Q15788_AD_840-1011
../outputs/AD_alphamissense_top10_variant_fasta/ERG_P11308_AD_118-261
../outputs/AD_alphamissense_top10_variant_fasta/CAMTA2_O94983_AD_472-581
../outputs/AD_alphamissense_top10_variant_fasta/ERG_P11308_AD_433-479
../outputs/AD_alphamissense_top10_variant_fasta/OTX1_P32242_AD_172-354
../outputs/AD_alphamissense_top10_variant_fasta/NCOA1_Q15788_AD_1241-1385
../outputs/AD_alphamissense_top10_variant_fasta/CAMTA2_O94983_AD_285-468
../outputs/AD_alphamissense_top10_variant_fasta/IKZF1_Q13422_AD_284-365
../outputs/AD_alphamissense_top10_variant_fasta/NKX2-2_O95096_AD_220-273
../outputs/AD_alphamissense_top10_variant_fasta/PAX6_P26367_AD_271-422
../outputs/AD_alphamissense_top10_variant_fasta/NR4A2_P43354_AD_584-598
../outputs/AD_alphamissense_top10_variant_fasta/MEIS2_O14770_AD_340-477
../outputs/AD_alphamissense_top10_variant_fasta/PITX1_P78337_AD_234-283
../outputs/AD_alphamissense_top10_variant_fasta/NR4A2_P43354_AD_

Now, repeating for bottom 10

In [51]:
bottom_10_am = pd.read_csv("../../output/am_10_least_pathogenic_15ADs.csv")
bottom_10_am

,Unnamed: 0,filepath,gene,uniprotID,start,end,ENST_x,Gene,range,foreign_key,...,ALT,genome,uniprot_id,transcript_id,protein_variant,am_pathogenicity,am_class,ENST_y,AA_pos,AA_orig
0,240,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,256,O95096_256,...,C,hg38,O95096,ENST00000377142.5,T256S,0.0635,likely_benign,ENST00000377142,256.0,T
1,274,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,261,O95096_261,...,A,hg38,O95096,ENST00000377142.5,T261S,0.0628,likely_benign,ENST00000377142,261.0,T
2,187,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,248,O95096_248,...,C,hg38,O95096,ENST00000377142.5,N248S,0.0627,likely_benign,ENST00000377142,248.0,N
3,290,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,264,O95096_264,...,T,hg38,O95096,ENST00000377142.5,P264H,0.0598,likely_benign,ENST00000377142,264.0,P
4,247,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,257,O95096_257,...,C,hg38,O95096,ENST00000377142.5,P257A,0.0583,likely_benign,ENST00000377142,257.0,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,186,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,202,P32242_202,...,G,hg38,P32242,ENST00000282549.7,N202S,0.0480,likely_benign,ENST00000282549,202.0,N
146,670,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,276,P32242_276,...,C,hg38,P32242,ENST00000282549.7,H276P,0.0471,likely_benign,ENST00000282549,276.0,H
147,821,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,297,P32242_297,...,C,hg38,P32242,ENST00000282549.7,H297P,0.0466,likely_benign,ENST00000282549,297.0,H
148,722,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,283,P32242_283,...,C,hg38,P32242,ENST00000282549.7,H283P,0.0439,likely_benign,ENST00000282549,283.0,H


In [53]:
bottom_10_am[22] = "No-Syn"
bottom_10_am[3] = "AD"
bottom_10_am[2] = bottom_10_am["POS"].astype(int)
bottom_10_am[17] = bottom_10_am["REF"]
bottom_10_am[18] = bottom_10_am["ALT"]
bottom_10_am

,Unnamed: 0,filepath,gene,uniprotID,start,end,ENST_x,Gene,range,foreign_key,...,am_pathogenicity,am_class,ENST_y,AA_pos,AA_orig,22,3,2,17,18
0,240,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,256,O95096_256,...,0.0635,likely_benign,ENST00000377142,256.0,T,No-Syn,AD,21511978,G,C
1,274,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,261,O95096_261,...,0.0628,likely_benign,ENST00000377142,261.0,T,No-Syn,AD,21511964,T,A
2,187,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,248,O95096_248,...,0.0627,likely_benign,ENST00000377142,248.0,N,No-Syn,AD,21512002,T,C
3,290,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,264,O95096_264,...,0.0598,likely_benign,ENST00000377142,264.0,P,No-Syn,AD,21511954,G,T
4,247,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,257,O95096_257,...,0.0583,likely_benign,ENST00000377142,257.0,P,No-Syn,AD,21511976,G,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,186,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,202,P32242_202,...,0.0480,likely_benign,ENST00000282549,202.0,N,No-Syn,AD,63055856,A,G
146,670,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,276,P32242_276,...,0.0471,likely_benign,ENST00000282549,276.0,H,No-Syn,AD,63056078,A,C
147,821,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,297,P32242_297,...,0.0466,likely_benign,ENST00000282549,297.0,H,No-Syn,AD,63056141,A,C
148,722,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,283,P32242_283,...,0.0439,likely_benign,ENST00000282549,283.0,H,No-Syn,AD,63056099,A,C


In [54]:
bottom_10_am = pd.merge(bottom_10_am, addit_info, on = "uniprotID", how = "left")
bottom_10_am

,Unnamed: 0,filepath,gene,uniprotID,start,end,ENST_x,Gene,range,foreign_key,...,ENST_y,AA_pos,AA_orig,22,3,2,17,18,0,13
0,240,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,256,O95096_256,...,ENST00000377142,256.0,T,No-Syn,AD,21511978,G,C,20,-
1,274,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,261,O95096_261,...,ENST00000377142,261.0,T,No-Syn,AD,21511964,T,A,20,-
2,187,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,248,O95096_248,...,ENST00000377142,248.0,N,No-Syn,AD,21512002,T,C,20,-
3,290,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,264,O95096_264,...,ENST00000377142,264.0,P,No-Syn,AD,21511954,G,T,20,-
4,247,../output/caitlin_experiment/variant fastas/NK...,NKX2-2,O95096,220,273,ENST00000377142,NKX2-2,257,O95096_257,...,ENST00000377142,257.0,P,No-Syn,AD,21511976,G,C,20,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,186,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,202,P32242_202,...,ENST00000282549,202.0,N,No-Syn,AD,63055856,A,G,2,+
146,670,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,276,P32242_276,...,ENST00000282549,276.0,H,No-Syn,AD,63056078,A,C,2,+
147,821,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,297,P32242_297,...,ENST00000282549,297.0,H,No-Syn,AD,63056141,A,C,2,+
148,722,../output/caitlin_experiment/variant fastas/OT...,OTX1,P32242,172,354,ENST00000282549,OTX1,283,P32242_283,...,ENST00000282549,283.0,H,No-Syn,AD,63056099,A,C,2,+


In [60]:
for uniprotID in set(bottom_10_am["uniprotID"]):
    print(uniprotID)
    variants_df = bottom_10_am[bottom_10_am["uniprotID"] == uniprotID]
    #display(variants_df)
    save_variant_fasta(uniprotID, variant_file_name = "alphamissense_bottom10", save = True, variants_df = variants_df)

O14770
../outputs/mutations/domains_alphamissense_bottom10_snv_classified/ENST00000561208.bed
ENST00000561208


,Unnamed: 0,filepath,gene,uniprotID,start,end,ENST_x,Gene,range,foreign_key,...,ENST_y,AA_pos,AA_orig,22,3,2,17,18,0,13
40,714,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,446,O14770_446,...,ENST00000561208,446.0,T,No-Syn,AD,36892271,T,G,15,-
41,713,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,446,O14770_446,...,ENST00000561208,446.0,T,No-Syn,AD,36892271,T,C,15,-
42,518,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,417,O14770_417,...,ENST00000561208,417.0,T,No-Syn,AD,36892358,T,C,15,-
43,464,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,409,O14770_409,...,ENST00000561208,409.0,P,No-Syn,AD,36892382,G,C,15,-
44,493,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,413,O14770_413,...,ENST00000561208,413.0,T,No-Syn,AD,36892370,T,G,15,-
45,707,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,445,O14770_445,...,ENST00000561208,445.0,P,No-Syn,AD,36892274,G,C,15,-
46,773,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,455,O14770_455,...,ENST00000561208,455.0,Q,No-Syn,AD,36892243,T,G,15,-
47,309,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,386,O14770_386,...,ENST00000561208,386.0,S,No-Syn,AD,36892451,T,C,15,-
48,848,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,466,O14770_466,...,ENST00000561208,466.0,N,No-Syn,AD,36892210,T,G,15,-
49,847,../output/caitlin_experiment/variant fastas/ME...,MEIS2,O14770,340,477,ENST00000561208,MEIS2,466,O14770_466,...,ENST00000561208,466.0,N,No-Syn,AD,36892210,T,C,15,-


'MAQRYDELPHYGGMDGVGVPASMYGDPHAPRPIPPVHHLNHGPPLHATQHYGAHAPHPNVMPASMGSAVNDALKRDKDAIYGHPLFPLLALVFEKCELATCTPREPGVAGGDVCSSDSFNEDIAVFAKQVRAEKPLFSSNPELDNLMIQAIQVLRFHLLELEKVHELCDNFCHRYISCLKGKMPIDLVIDERDGSSKSDHEELSGSSTNLADHNPSSWRDHDDATSTHSAGTPGPSSGGHASQSGDNSSEQGDGLDNSVASPGTGDDDDPDKDKKRQKKRGIFPKVATNIMRAWLFQHLTHPYPSEEQKKQLAQDTGLTILQVNNWFINARRRIVQPMIDQSNRAGFLLDPSVSQGAAYSPEGQPMGSFVLDGQQHMGIRPAGLQSMPGDYVSQGGPMGMSMAQPSYTPPQMTPHPTQLRHGPPMHSYLPSHPHHPAMMMHGGPPTHPGMTMSAQSPTMLNSVDPNVGGQVMDIHAQ*'

,nt,pred_prot_pos
gen_pos,,
37099466,T,1
37099465,A,1
37099464,C,1
37099463,C,2
37099462,G,2
...,...,...
36892177,T,477
36892176,T,477
36892175,A,478


340
477


...

,index,name,TF_seq,prot_pos,AD_seq


1241
1385


,index,name,TF_seq,prot_pos,AD_seq
0,0,g.2.24758004A>G(p.Thr1305Ala),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1305,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
1,1,g.2.24752062C>G(p.Pro1263Ala),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1263,GEANFAPSLSPGSSMVPMPIPPAQSSLLQQTPPASGYQSPDMKAWQ...
2,2,g.2.24752089C>G(p.Pro1272Ala),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1272,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTAPASGYQSPDMKAWQ...
3,3,g.2.24758007C>G(p.Pro1306Ala),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1306,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
4,4,g.2.24758148C>T(p.Pro1353Ser),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1353,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
5,5,g.2.24762709A>C(p.His1363Pro),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1363,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
6,6,g.2.24758097G>T(p.Ala1336Ser),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1336,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
7,7,g.2.24752092C>G(p.Pro1273Ala),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1273,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPAASGYQSPDMKAWQ...
8,8,g.2.24752145A>G(p.Ile1290Met),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1290,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...
9,9,g.2.24758098C>G(p.Ala1336Gly),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,1336,GEANFAPSLSPGSSMVPMPIPPPQSSLLQQTPPASGYQSPDMKAWQ...


840
1011


,index,name,TF_seq,prot_pos,AD_seq
0,10,g.2.24729552C>G(p.Pro980Ala),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,980,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
1,11,g.2.24711033A>C(p.Thr841Pro),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,841,VPSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
2,12,g.2.24729537C>G(p.Pro975Ala),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,975,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
3,13,g.2.24726686A>G(p.Ile899Met),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,899,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
4,14,g.2.24726699T>C(p.Ser904Pro),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,904,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
5,15,g.2.24726694G>A(p.Ser902Asn),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,902,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
6,16,g.2.24711036A>G(p.Ser842Gly),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,842,VTGVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
7,17,g.2.24729561A>C(p.Ile983Leu),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,983,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
8,18,g.2.24729606C>G(p.Pro998Ala),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,998,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...
9,19,g.2.24729563C>G(p.Ile983Met),MSGLGDSSSDPANPDSHKRKGSPCDTLASSTEKRRREQENKYLEEL...,983,VTSVTIKSEILPASLQSATARPTSRLNRLPELELEAIDNQFGQPGT...


In [61]:
# Checking work - distance at most 1 from known AD?

variant_ads = glob.glob("../outputs/AD_alphamissense_bottom10_variant_fasta/*")

for variant_AD in variant_ads:
    print(variant_AD)
    # Extracting info about AD (name and position)
    AD_descrip = variant_AD.split("/")[-1]
    gene_name = AD_descrip.split("_")[0]
    coords = AD_descrip.split("_")[-1]
    start = int(coords.split("-")[0])
    end = int(coords.split("-")[1])
    
    # Extracting corresponding wt AD seq
    entry = known_ADs[(known_ADs["Gene"] == gene_name) & (known_ADs["Start"] == start) & (known_ADs["End"] == end)]
    if len(entry) == 0:
        print("ERROR: AD not found")
    else:
        wt_seq = entry["ProteinRegionSeq"].iloc[0]
        
        # Finding variant sequences from fasta
        for var_seq in list(protfasta.read_fasta(variant_AD).values()):
            # Comparing distance
            dist = Levenshtein.distance(wt_seq, var_seq)
            #print(wt_seq)
            #print(first_var)
            
            if (dist != 1):
                print("DIFF > 1")
                print(dist)
                #print(wt_seq)
                #print(var_seq)
                print("----")

# As expected, only ERG different.

../outputs/AD_alphamissense_bottom10_variant_fasta/NCOA1_Q15788_AD_840-1011
../outputs/AD_alphamissense_bottom10_variant_fasta/ERG_P11308_AD_118-261
../outputs/AD_alphamissense_bottom10_variant_fasta/CAMTA2_O94983_AD_472-581
../outputs/AD_alphamissense_bottom10_variant_fasta/ERG_P11308_AD_433-479
../outputs/AD_alphamissense_bottom10_variant_fasta/OTX1_P32242_AD_172-354
../outputs/AD_alphamissense_bottom10_variant_fasta/NCOA1_Q15788_AD_1241-1385
../outputs/AD_alphamissense_bottom10_variant_fasta/CAMTA2_O94983_AD_285-468
../outputs/AD_alphamissense_bottom10_variant_fasta/IKZF1_Q13422_AD_284-365
../outputs/AD_alphamissense_bottom10_variant_fasta/NKX2-2_O95096_AD_220-273
../outputs/AD_alphamissense_bottom10_variant_fasta/PAX6_P26367_AD_271-422
../outputs/AD_alphamissense_bottom10_variant_fasta/NR4A2_P43354_AD_584-598
../outputs/AD_alphamissense_bottom10_variant_fasta/MEIS2_O14770_AD_340-477
../outputs/AD_alphamissense_bottom10_variant_fasta/PITX1_P78337_AD_234-283
../outputs/AD_alphamissen

In [67]:
for uniprotID in list(files):
    save_variant_fasta(uniprotID, variant_file_name = "clinvar_for_15", save = True)

../outputs/mutations/domains_clinvar_for_15_snv_classified/ENST00000592199.bed
ENST00000592199  has no variants.
../outputs/mutations/domains_clinvar_for_15_snv_classified/ENST00000561208.bed
ENST00000561208


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,15,36896671,36896672,DBD,ENSG00000134138,.,-,ENST00000561208,15,36896671,...,-,15,36896671,36896672,C,T,.,R,K,No-Syn
1,15,36896676,36896677,DBD,ENSG00000134138,.,-,ENST00000561208,15,36896676,...,-,15,36896676,36896677,A,T,.,N,K,No-Syn
2,15,36950327,36950328,DBD,ENSG00000134138,.,-,ENST00000561208,15,36950327,...,-,15,36950327,36950328,T,C,.,N,D,No-Syn
3,15,36950332,36950333,DBD,ENSG00000134138,.,-,ENST00000561208,15,36950332,...,-,15,36950332,36950333,A,C,.,V,G,No-Syn
4,15,36950335,36950336,DBD,ENSG00000134138,.,-,ENST00000561208,15,36950335,...,-,15,36950335,36950336,T,A,.,Q,L,No-Syn
5,15,36950336,36950337,DBD,ENSG00000134138,.,-,ENST00000561208,15,36950336,...,-,15,36950336,36950337,G,C,.,Q,E,No-Syn
6,15,36950384,36950385,DBD,ENSG00000134138,.,-,ENST00000561208,15,36950384,...,-,15,36950384,36950385,C,T,.,E,K,No-Syn
7,15,36950395,36950396,DBD,ENSG00000134138,.,-,ENST00000561208,15,36950395,...,-,15,36950395,36950396,G,A,.,P,L,No-Syn
8,15,36950397,36950398,DBD,ENSG00000134138,.,-,ENST00000561208,15,36950397,...,-,15,36950397,36950398,A,T,.,H,Q,No-Syn
9,15,37036836,37036837,DBD,ENSG00000134138,.,-,ENST00000561208,15,37036836,...,-,15,37036836,37036837,C,A,.,A,S,No-Syn


'MAQRYDELPHYGGMDGVGVPASMYGDPHAPRPIPPVHHLNHGPPLHATQHYGAHAPHPNVMPASMGSAVNDALKRDKDAIYGHPLFPLLALVFEKCELATCTPREPGVAGGDVCSSDSFNEDIAVFAKQVRAEKPLFSSNPELDNLMIQAIQVLRFHLLELEKVHELCDNFCHRYISCLKGKMPIDLVIDERDGSSKSDHEELSGSSTNLADHNPSSWRDHDDATSTHSAGTPGPSSGGHASQSGDNSSEQGDGLDNSVASPGTGDDDDPDKDKKRQKKRGIFPKVATNIMRAWLFQHLTHPYPSEEQKKQLAQDTGLTILQVNNWFINARRRIVQPMIDQSNRAGFLLDPSVSQGAAYSPEGQPMGSFVLDGQQHMGIRPAGLQSMPGDYVSQGGPMGMSMAQPSYTPPQMTPHPTQLRHGPPMHSYLPSHPHHPAMMMHGGPPTHPGMTMSAQSPTMLNSVDPNVGGQVMDIHAQ*'

,nt,pred_prot_pos
gen_pos,,
37099466,T,1
37099465,A,1
37099464,C,1
37099463,C,2
37099462,G,2
...,...,...
36892177,T,477
36892176,T,477
36892175,A,478


340
477


...

1
91


,index,name,TF_seq,prot_pos,AD_seq
0,0,g.2.156330029G>A(p.Ala53Val),MPCVQAQYGSSPQGASPASQSYSYHSSGEYSSDFLTPEFVKFSMDL...,53,MPCVQAQYGSSPQGASPASQSYSYHSSGEYSSDFLTPEFVKFSMDL...
1,1,g.2.156330108A>G(p.Ser27Pro),MPCVQAQYGSSPQGASPASQSYSYHSPGEYSSDFLTPEFVKFSMDL...,27,MPCVQAQYGSSPQGASPASQSYSYHSPGEYSSDFLTPEFVKFSMDL...


584
598


,index,name,TF_seq,prot_pos,AD_seq
0,2,g.2.156325777A>C(p.Ile588Met),MPCVQAQYGSSPQGASPASQSYSYHSSGEYSSDFLTPEFVKFSMDL...,588,PPAIMDKLFLDTLPF


../outputs/mutations/domains_clinvar_for_15_snv_classified/ENST00000437473.bed
ENST00000437473  has no variants.
../outputs/mutations/domains_clinvar_for_15_snv_classified/ENST00000317216.bed
ENST00000317216  has no variants.
../outputs/mutations/domains_clinvar_for_15_snv_classified/ENST00000428368.bed
ENST00000428368  has no variants.


In [69]:
# Checking work - distance at most 1 from known AD?

variant_ads = glob.glob("../outputs/AD_alphamissense_top10_variant_fasta/*")

for variant_AD in variant_ads:
    print(variant_AD)
    # Extracting info about AD (name and position)
    AD_descrip = variant_AD.split("/")[-1]
    gene_name = AD_descrip.split("_")[0]
    coords = AD_descrip.split("_")[-1]
    start = int(coords.split("-")[0])
    end = int(coords.split("-")[1])
    
    # Extracting corresponding wt AD seq
    entry = known_ADs[(known_ADs["Gene"] == gene_name) & (known_ADs["Start"] == start) & (known_ADs["End"] == end)]
    if len(entry) == 0:
        print("ERROR: AD not found")
    else:
        wt_seq = entry["ProteinRegionSeq"].iloc[0]
        
        # Finding variant sequences from fasta
        for var_seq in list(protfasta.read_fasta(variant_AD).values()):
            # Comparing distance
            dist = Levenshtein.distance(wt_seq, var_seq)
            #print(wt_seq)
            #print(first_var)
            
            if (dist != 1):
                print("DIFF > 1")
                print(dist)
                #print(wt_seq)
                #print(var_seq)
                print("----")



../outputs/AD_alphamissense_top10_variant_fasta/NCOA1_Q15788_AD_840-1011
../outputs/AD_alphamissense_top10_variant_fasta/ERG_P11308_AD_118-261
../outputs/AD_alphamissense_top10_variant_fasta/CAMTA2_O94983_AD_472-581
../outputs/AD_alphamissense_top10_variant_fasta/ERG_P11308_AD_433-479
../outputs/AD_alphamissense_top10_variant_fasta/OTX1_P32242_AD_172-354
../outputs/AD_alphamissense_top10_variant_fasta/NCOA1_Q15788_AD_1241-1385
../outputs/AD_alphamissense_top10_variant_fasta/CAMTA2_O94983_AD_285-468
../outputs/AD_alphamissense_top10_variant_fasta/IKZF1_Q13422_AD_284-365
../outputs/AD_alphamissense_top10_variant_fasta/NKX2-2_O95096_AD_220-273
../outputs/AD_alphamissense_top10_variant_fasta/PAX6_P26367_AD_271-422
../outputs/AD_alphamissense_top10_variant_fasta/NR4A2_P43354_AD_584-598
../outputs/AD_alphamissense_top10_variant_fasta/MEIS2_O14770_AD_340-477
../outputs/AD_alphamissense_top10_variant_fasta/PITX1_P78337_AD_234-283
../outputs/AD_alphamissense_top10_variant_fasta/NR4A2_P43354_AD_

In [137]:
for uniprotID in list(files):   
    save_variant_fasta(uniprotID, variant_file_name = "south_asian", save = True)

../outputs/mutations/domains_south_asian_snv_classified/ENST00000592199.bed
ENST00000592199


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,19,13087994,13087995,AD,ENSG00000008441,.,+,ENST00000592199,19,13087994,...,+,19,13087994,13087995,G,A,0.000187,G,S,No-Syn
1,19,13088100,13088101,AD,ENSG00000008441,.,+,ENST00000592199,19,13088100,...,+,19,13088100,13088101,C,G,0.000187,A,G,No-Syn


'MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEKRMSKDEERAVKDELLGEKPEIKQKWASRLLAKLRKDIRPEFREDFVLTITGKKPPCCVLSNPDQKGKIRRIDCLRQADKVWRLDLVMVILFKGIPLESTDGERLYKSPQCSNPGLCVQPHHIGVTIKELDLYLAYFVHTPESGQSDSSNQQGDADIKPLPNGHLSFQDCFVTSGVWNVTELVRVSQTPVATASGPNFSLADLESPSYYNINQVTLGRRSITSPPSTSTTKRPKSIDDSEMESPVDDVFYPGTGRSPAAGSSQSSGWPNDVDAGPASLKKSGKLDFCSALSSQGSSPRMAFTHHPLPVLAGVRPGSPRATASALHFPSTSIIQQSSPYFTHPTIRYHHHHGQDSLKEFVQFVCSDGSGQATGQPNGSGQGKVPGSFLLPPPPPVARPVPLPMPDSKSTSTAPDGAALTPPSPSFATTGASSANRFVSIGPRDGNFLNIPQQSQSWFL*'

,nt,pred_prot_pos
gen_pos,,
12995838,A,1
12995839,T,1
12995840,G,1
12995841,T,2
12995842,A,2
...,...,...
13094645,T,502
13094646,C,502
13094647,T,503


414
502


...

209
252


,index,name,TF_seq,prot_pos,AD_seq
0,0,g.19.1854509C>T(p.Gly237Arg),MSAAVACVDYFAADVLMAISSGAVVHRGRPGPEGAGPAAGLDVRAA...,237,HPGFHPDLLRRPGARSTSPSDSLPCSLARSPAPSPAPSPAPAGL
1,1,g.19.1854550C>T(p.Arg223His),MSAAVACVDYFAADVLMAISSGAVVHRGRPGPEGAGPAAGLDVRAA...,223,HPGFHPDLLRRPGAHSTSPSDSLPCSLAGSPAPSPAPSPAPAGL


../outputs/mutations/domains_south_asian_snv_classified/ENST00000395324.bed
ENST00000395324


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,12,47865103,47865104,DBD,ENSG00000111424,.,-,ENST00000395324,12,47865103,...,-,12,47865103,47865104,G,A,0.000187,R,C,No-Syn
1,12,47865133,47865134,DBD,ENSG00000111424,.,-,ENST00000395324,12,47865133,...,-,12,47865133,47865134,C,T,0.000187,G,R,No-Syn
2,12,47879024,47879025,DBD,ENSG00000111424,.,-,ENST00000395324,12,47879024,...,-,12,47879024,47879025,C,T,0.000187,R,Q,No-Syn


IndexError: single positional indexer is out-of-bounds

In [139]:
# Checking work - distance at most 1 from known AD?

variant_ads = glob.glob("../outputs/AD_south_asian_variant_fasta/*")

for variant_AD in variant_ads:
    print(variant_AD)
    # Extracting info about AD (name and position)
    AD_descrip = variant_AD.split("/")[-1]
    gene_name = AD_descrip.split("_")[0]
    coords = AD_descrip.split("_")[-1]
    start = int(coords.split("-")[0])
    end = int(coords.split("-")[1])
    
    # Extracting corresponding wt AD seq
    entry = known_ADs[(known_ADs["Gene"] == gene_name) & (known_ADs["Start"] == start) & (known_ADs["End"] == end)]
    if len(entry) == 0:
        print("ERROR: AD not found")
    else:
        wt_seq = entry["ProteinRegionSeq"].iloc[0]
        
        # Finding variant sequences from fasta
        for var_seq in list(protfasta.read_fasta(variant_AD).values()):
            # Comparing distance
            dist = Levenshtein.distance(wt_seq, var_seq)
            #print(wt_seq)
            #print(first_var)
            
            if (dist != 1):
                print("DIFF > 1")
                print(dist)
                #print(wt_seq)
                #print(var_seq)
                print("----")



../outputs/AD_south_asian_variant_fasta/RORA_P35398_AD_272-385
../outputs/AD_south_asian_variant_fasta/THRA_P10827_AD_1-52
../outputs/AD_south_asian_variant_fasta/YY1_P25490_AD_371-397
../outputs/AD_south_asian_variant_fasta/NCOA1_Q15788_AD_840-1011
../outputs/AD_south_asian_variant_fasta/ERG_P11308_AD_118-261
../outputs/AD_south_asian_variant_fasta/CAMTA2_O94983_AD_472-581
../outputs/AD_south_asian_variant_fasta/ERG_P11308_AD_433-479
../outputs/AD_south_asian_variant_fasta/EBF3_Q9H4W6_AD_281-360
../outputs/AD_south_asian_variant_fasta/YY1_P25490_AD_1-69
../outputs/AD_south_asian_variant_fasta/MTF1_Q14872_AD_510-624
../outputs/AD_south_asian_variant_fasta/OTX1_P32242_AD_172-354
../outputs/AD_south_asian_variant_fasta/MTF1_Q14872_AD_328-509
../outputs/AD_south_asian_variant_fasta/NCOA1_Q15788_AD_1241-1385
../outputs/AD_south_asian_variant_fasta/CAMTA2_O94983_AD_285-468
../outputs/AD_south_asian_variant_fasta/KMT2A-KMT2A-ALL1-HRX-KMT2A-MLL1-CXXC7_Q03164_AD_2782-2921
ERROR: AD not found
.